## Installing Haystack


In [ ]:

pip install --upgrade pip
pip install farm-haystack[colab,faiss]==1.17.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for farm-haystack[colab,faiss]==1.17.2 from https://files.pythonhosted.org/packages/01/22/b5ae071c0efc32502ee5c5aea0983ac3f105c41f693b5199aa42c664247e/farm_haystack-1.17.2-py3-none-any.whl.metadata
  Obtaining dependency information for azure-ai-formrecognizer>=3.2.0b2 from https://files.pythonhosted.org/packages/ec/d6/255f4afed1e4d0e2aaf1b988d4c5ae453b65d9b92a3370634d7523804fd7/azure_ai_formrecognizer-3.3.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 3.1 MB/s eta 0:00:00
  Obtaining dependency information for canals==0.2.2 from https://files.pythonhosted.org/packages/5c/b8/3e878efd413272a351228874b9816ec346223eee97521ba58c5790ac237a/canals-0.2.2-py3-none-any.whl.metadata
  Obtaining dependenc

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.
tensorflow 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Initializing the DocumentStore

In [ ]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 0s (2,107 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120901 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.2_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.2) ...
Setting up poppler-utils (22.02.0-2ubuntu0.2) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# Initialize a PDFToTextConverter to convert the PDF to text
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PDFToTextConverter, PreProcessor, DensePassageRetriever
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor

In [ ]:
pdf_paths = ["/content/DPM2009.pdf", "/content/DFPDS21.pdf"]
#doc_dir = "data/document"

# Convert the PDF documents to Haystack-compatible format
pdf_converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
pdf_docs = []

for pdf_path in pdf_paths:
    doc = pdf_converter.convert(file_path=pdf_path, meta=None)[0]
    pdf_docs.append(doc)

In [ ]:
# Initializing the PreProcessor to clean and split the text
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
processed_docs = preprocessor.process(pdf_docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Preprocessing:   0%|          | 0/2 [00:00<?, ?docs/s]

In [ ]:
processed_docs

[<Document: {'content': '\x0c\x0c\x0cINDEX\nS. NO. CONTENTS PAGE\nNO.\nCHAPTERS\n2. Chapter-2 : PROCUREMENT – OBJECTIVE AND\nPOLICY\n5. Chapter-5: APPROVAL PROCESS AND\nCONCLUSION OF CONTRACT\n9. Chapter-9 : PROCUREMENT OF GOODS AND\nSERVICES FROM FOREIGN COUNTRIES\n10. Chapter-10. : STANDARD CONDITIONS OF\nCONTRACT - FOREIGN PROCUREMENT\n11. Chapter-11 : REPAIR CONTRACTS WITH FOREIGN\nAND INDIGENOUS FIRMS\n13. Chapter-13 : EVALUATION OF QUOTATIONS AND\nPRICE REASONABILITY\n14. Chapter-14 : OFFLOADING OF PARTIAL/ COMPLETE\nREFITS/REPAIRS OF SHIPS/\nSUBMARINES/CRAFTS/ASSETS TO INDIAN\nPSU/PRIVATE SHIP YARDS/TRADE\n15. Chapter-15 : DESIGN, DEVELOPMENT AND\nFABRICATION CONTRACTS\x0cAPPENDICES\n16. Appendix – A\nTIME FRAME FOR PROCUREMENT\n17. ', 'content_type': 'text', 'score': None, 'meta': {'_split_id': 0}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '1497e709bbce4476d9f1a817af572179'}>,
 <Document: {'content': 'Appendix- B\nFORMAT FOR THE STATEMENT OF CASE (SOC)\n18. Appendix

## Initializing the Retriever

In [ ]:
# Initialize a DensePassageRetriever
document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat")
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
    passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english


INFO:haystack.modeling.model.language_model:Auto-detected model language: english


In [ ]:
document_store.write_documents(processed_docs)
document_store.update_embeddings(retriever)

Writing Documents:   0%|          | 0/1874 [00:00<?, ?it/s]

INFO:haystack.document_stores.faiss:Updating embeddings for 1840 docs...


Updating Embedding:   0%|          | 0/1840 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/1840 [00:00<?, ? Docs/s]

## Initializing the Generator

In [ ]:
from haystack.nodes import Seq2SeqGenerator


generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


## Initializing the Pipeline



In [ ]:
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)

## Ask Questions

In [ ]:
piperun=pipe.run(
    query="How should a single tender inquiry be undertaken?", params={"Retriever": {"top_k": 15}}
)
piperun['answers'][0].answer

' A tender inquiry is a process by which a government agency solicits bids for a specific service. The idea is that the bids will be evaluated by an independent third party, who will then make a decision on whether or not to accept the bid.'

In [ ]:
piperun=pipe.run(
    query="under what conditions should single tender inquiry should undertaken?", params={"Retriever": {"top_k": 5}}
)
piperun['answers'][0].answer

" Single Tender inquiries should not be undertaken. They should be conducted by an independent third party. If you don't have a third party, you can't conduct a tender inquiry."

In [ ]:
piperun=pipe.run(
    query="What is the Fall Clause in the contract, and how does it affect pricing?", params={"Retriever": {"top_k": 5}}
)
piperun['answers'][0].answer

' The Fall Clause says that if you sell a contract for a price that is lower than the rate contract price, you will automatically be charged that price for the subsequent supplies under that rate contract.'

In [ ]:
piperun=pipe.run(
    query="What is the short title of the DPM manual and when did it come into force?", params={"Retriever": {"top_k":2}}
)
piperun['answers'][0].answer

" URL_0  It's the short title of the Department of Defense Manual."

In [ ]:
piperun=pipe.run(
    query="What does the term Indent mean in the context of procurement?", params={"Retriever": {"top_k": 2}}
)
piperun['answers'][0].answer

' An indent is a requisition placed by the procurement authority on the procurement agency to allow prompt procurement of the item.'

In [ ]:

piperun=pipe.run(
    query="Which organizations are required to follow the procedures laid down in the manual?", params={"Retriever": {"top_k":3}}
)
piperun['answers'][0].answer

' They are required to follow the procedures laid down in the manual.'

In [ ]:

piperun=pipe.run(
    query="What is the scope of the term Procurement in the DPM?", params={"Retriever": {"top_k": 3}}
)
piperun['answers'][0].answer

' Procurement is the process of buying goods and services from a third party.'

In [ ]:

piperun=pipe.run(
    query="What is full form of DPM in manual?", params={"Retriever": {"top_k":2}}
)
piperun['answers'][0].answer

' The full form of a DPM is the one that is printed on the manual.'

In [ ]:
piperun=pipe.run(
    query="What do the abbreviations GSQRs, JSQRs, NSQRs, and AFSQRs stand for?", params={"Retriever": {"top_k": 6}}
)
piperun['answers'][0].answer

' GSQR stands for General Staff Qualification. It is the highest rank a person can be in the military. JSQRs stands for Joint Special Qualification, which is the lowest rank an individual can be. AFSQR is an acronym for Armed Forces Qualification Review.'

In [ ]:
piperun=pipe.run(
    query="How do GSQRs reflect the user's requirements for equipment?", params={"Retriever": {"top_k": 3}}
)
piperun['answers'][0].answer

'A GSQR is a list of requirements for a specific piece of equipment. For example, if you want to build a machine gun, you need to be able to build it with a specific set of specifications. A JSFQR (Joint Staff Qualitative Requirements) is a set of requirements that the Joint Chiefs of Staff (JCS) use to determine what types of equipment they want to buy. The JCS is a collection of specifications that the JCS uses to decide what kinds of equipment to buy for each branch of the military.'

In [ ]:

piperun=pipe.run(
    query="What does the term ruggedized mean in the context of these requirements?", params={"Retriever": {"top_k": 3}}
)
piperun['answers'][0].answer

'The term "ruggedized" is used in the military to refer to an item that has been modified to be more rugged than it would have otherwise been. For example, if you were to take a piece of armor and put it on a motorcycle, it would not be as rugged as it would be if you took it off a motorcycle.'